# Predicting the number of air passengers

<i> Aya Errajraji, Khouloud El Alami </i>

In [1]:
import os
import numpy as np
import pandas as pd
import holidays
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
# Downloading the external_data given in the starting kit
__file__ = os.path.join('submissions', 'starting_kit', 'estimator.py')
filepath = os.path.join(os.path.dirname(__file__), 'external_data.csv')
df = pd.read_csv(filepath, parse_dates=["Date"])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11040 entries, 0 to 11039
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date                        11040 non-null  datetime64[ns]
 1   AirPort                     11040 non-null  object        
 2   Max TemperatureC            11040 non-null  int64         
 3   Mean TemperatureC           11040 non-null  int64         
 4   Min TemperatureC            11040 non-null  int64         
 5   Dew PointC                  11040 non-null  int64         
 6   MeanDew PointC              11040 non-null  int64         
 7   Min DewpointC               11040 non-null  int64         
 8   Max Humidity                11040 non-null  int64         
 9   Mean Humidity               11040 non-null  int64         
 10  Min Humidity                11040 non-null  int64         
 11  Max Sea Level PressurehPa   11040 non-null  int64     

In [4]:
df.isna().sum()

Date                             0
AirPort                          0
Max TemperatureC                 0
Mean TemperatureC                0
Min TemperatureC                 0
Dew PointC                       0
MeanDew PointC                   0
Min DewpointC                    0
Max Humidity                     0
Mean Humidity                    0
Min Humidity                     0
Max Sea Level PressurehPa        0
Mean Sea Level PressurehPa       0
Min Sea Level PressurehPa        0
Max VisibilityKm                 0
Mean VisibilityKm                0
Min VisibilitykM                 0
Max Wind SpeedKm/h               0
Mean Wind SpeedKm/h              0
Max Gust SpeedKm/h             290
Precipitationmm                  0
CloudCover                       0
Events                        6745
WindDirDegrees                   0
dtype: int64

In [5]:
# Choosing the data that seems more relevant for our prediction problem
external_data = df[['Date', 'AirPort', 'Max TemperatureC', 'MeanDew PointC', 'Max Humidity', 'WindDirDegrees']]

external_data = external_data.rename(columns={'Date': 'DateOfDeparture'})
external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees
0,2011-09-01,ATL,35,18,79,129
1,2011-09-02,ATL,36,15,61,185
2,2011-09-03,ATL,35,16,64,147
3,2011-09-04,ATL,27,19,93,139
4,2011-09-05,ATL,26,22,94,149


In [6]:
external_data['AirPort'].unique()

array(['ATL', 'ORD', 'LAX', 'DFW', 'DEN', 'JFK', 'SFO', 'CLT', 'LAS',
       'PHX', 'IAH', 'MIA', 'MCO', 'EWR', 'SEA', 'MSP', 'DTW', 'PHL',
       'BOS', 'LGA'], dtype=object)

In [7]:
# Importing geographical data to add the coordinates of each airport
df_geo = pd.read_csv('GlobalAirportDatabase.csv', sep=':', header=None)
cols = ['ICAO Code', 'AirPort', 'Airport Name', 'City/Town', 'Country', 'Latitude Degrees', 'Latitude Minutes',
        'Latitude Seconds', 'Latitude Direction', 'Longitude Degrees', 'Longitude Minutes', 'Longitude Seconds',
        'Longitude Direction', 'Altitude', 'Latitude', 'Longitude']
df_geo.columns = cols

# Only choosing the airports that we need 
df_geo = df_geo[df_geo["AirPort"].isin(['ATL', 'ORD', 'LAX', 'DFW', 'DEN', 'JFK', 'SFO', 'CLT', 'LAS',
       'PHX', 'IAH', 'MIA', 'MCO', 'EWR', 'SEA', 'MSP', 'DTW', 'PHL','BOS', 'LGA'])]

# Only keeping the columns that we're interested in
df_geo = df_geo[['AirPort', 'Latitude', 'Longitude']]
df_geo

,AirPort,Latitude,Longitude
3401,ATL,33.640,-84.427
3426,BOS,42.364,-71.005
3452,CLT,35.214,-80.943
3474,DEN,39.858,-104.667
3476,DFW,32.896,-97.037
3489,DTW,42.212,-83.353
3503,EWR,40.692,-74.169
3577,IAH,29.980,-95.340
3596,JFK,40.640,-73.779
3600,LAS,36.080,-115.152


In [8]:
# Merging the coordinates to our dataset
external_data = pd.merge(external_data, df_geo, how='left', on=['AirPort'], sort=False)
external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427


In [9]:
# Importing traffic data
df_traffic = pd.read_csv('PassengerTraffic.csv')
df_traffic

,year,AirPort,Traffic
0,2011,ATL,92389023
1,2011,ORD,66701241
2,2011,LAX,61862052
3,2011,DFW,57832495
4,2011,DEN,52849132
5,2011,JFK,47644060
6,2011,SFO,40927786
7,2011,CLT,39043708
8,2011,LAS,40560285
9,2011,PHX,40591948


In [10]:
# Creating a temporary column to have the year
external_data.loc[:, 'year'] = external_data['DateOfDeparture'].dt.year

# Adding the traffic to our dataset
external_data = pd.merge(external_data, df_traffic, how='left', on=['AirPort', 'year'], sort=False)

external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,year,Traffic
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427,2011,92389023
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427,2011,92389023
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427,2011,92389023
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427,2011,92389023
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427,2011,92389023


In [11]:
# Importing GDP data
df_gdp = pd.read_csv('GDB_by_state.csv')
df_gdp

,State,year,GDP
0,AZ,2011,38248
1,AZ,2012,38537
2,AZ,2013,38292
3,CA,2011,52067
4,CA,2012,52912
5,CA,2013,53746
6,CO,2011,49274
7,CO,2012,49622
8,CO,2013,50426
9,FL,2011,37859


In [12]:
# Creating a dictionary that maps airports by state
states = {'ATL' : 'GA',
          'ORD':'IL',
          'LAX':'CA',
          'DFW':'TX',
          'DEN':'CO',
          'JFK':'NY',
          'SFO':'CA',
          'CLT':'NC',
          'LAS':'NV',
          'PHX':'AZ',
          'IAH':'TX',
          'MIA':'FL',
          'MCO':'FL',
          'EWR':'NJ',
          'SEA':'WA',
          'MSP':'MN',
          'DTW':'MI',
          'PHL':'PA',
          'BOS':'MA',
          'LGA':'NY'}

# Creating a temporary column to have the state of each airport
external_data["State"] = external_data["AirPort"].map(states)

# Adding the GDP to our dataset
external_data = pd.merge(external_data, df_gdp, how='left', on=['State', 'year'], sort=False)

# Dropping the column we don't need anymore
external_data.drop("year", axis=1, inplace=True)

external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,State,GDP
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427,92389023,GA,42137
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427,92389023,GA,42137
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427,92389023,GA,42137
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427,92389023,GA,42137
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427,92389023,GA,42137


In [13]:
# Adding the Holidays per State

n = external_data.shape[0]
j = external_data.columns.get_loc("State")

external_data["is_State_Holiday"] = 0
k = external_data.columns.get_loc("is_State_Holiday")

for i in range(n):
    state = external_data.iloc[i, j]
    # Getting the list of holidays for the airport's state
    state_holidays = holidays.CountryHoliday('US', state=state)
    # Getting the date we're testing
    date = external_data.iloc[i,0]
    # Testing if the day is a holiday in the airport's state
    if date in state_holidays :
        external_data.iloc[i, k] = 1

# Dropping the column we don't need anymore
external_data.drop("State", axis=1, inplace=True)

external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,GDP,is_State_Holiday
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427,92389023,42137,0
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427,92389023,42137,0
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427,92389023,42137,0
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427,92389023,42137,0
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427,92389023,42137,1


In [14]:
# Computing the distance to the closest holiday (last or next)

# Creating an annexe dataframe to work on
df_holidays = external_data.loc[:,["DateOfDeparture", "is_State_Holiday"]]
df_holidays["Distance_To_Last_Holiday"] = 0
df_holidays["Distance_To_Next_Holiday"] = 0

# Distance to last holiday
for i in range(n):
    test = df_holidays.iloc[i, 1]
    index = i 
    while (index > 0) and (test == 0):
        index -= 1
        test = df_holidays.iloc[index, 1]
    if test : # If the loop was stopped because we found a holiday
        current_date = df_holidays.iloc[i, 0]
        last_holiday = df_holidays.iloc[index, 0]
        df_holidays.iloc[i, 2] = abs(last_holiday - current_date).days
    else : # If the loop was stopped because we reached the end of the dataset
        df_holidays.iloc[i, 2] = 365

# Distance to next holiday
for i in range(n):
    test = df_holidays.iloc[i, 1]
    index = i 
    while (index < (n - 1)) and (test == 0):
        index += 1
        test = df_holidays.iloc[index, 1]
    if test : # If the loop was stopped because we found a holiday
        current_date = df_holidays.iloc[i, 0]
        next_holiday = df_holidays.iloc[index, 0]
        df_holidays.iloc[i, 3] = abs(next_holiday - current_date).days   
    else : # If the loop was stopped because we reached the end of the dataset
        df_holidays.iloc[i, 3] = 365
        
# Taking the minimum of the two values
external_data['Distance_To_Closest_Holiday'] = pd.DataFrame([df_holidays["Distance_To_Last_Holiday"], df_holidays["Distance_To_Next_Holiday"]]).min()

external_data

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,GDP,is_State_Holiday,Distance_To_Closest_Holiday
0,2011-09-01,ATL,35,18,79,129,33.640,-84.427,92389023,42137,0,4
1,2011-09-02,ATL,36,15,61,185,33.640,-84.427,92389023,42137,0,3
2,2011-09-03,ATL,35,16,64,147,33.640,-84.427,92389023,42137,0,2
3,2011-09-04,ATL,27,19,93,139,33.640,-84.427,92389023,42137,0,1
4,2011-09-05,ATL,26,22,94,149,33.640,-84.427,92389023,42137,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,-3,76,320,40.777,-73.872,26722183,62605,0,11
11036,2013-03-02,LGA,4,-5,82,317,40.777,-73.872,26722183,62605,0,12
11037,2013-03-03,LGA,4,-8,69,314,40.777,-73.872,26722183,62605,0,13
11038,2013-03-04,LGA,5,-8,63,313,40.777,-73.872,26722183,62605,0,14


In [15]:
# Exploring USA's national holidays on 2011
for date, name in sorted(holidays.US(state='CO', years=2011).items()):
    print(date, name)

2010-12-31 New Year's Day (Observed)
2011-01-01 New Year's Day
2011-01-17 Martin Luther King Jr. Day
2011-02-21 Washington's Birthday
2011-05-30 Memorial Day
2011-07-04 Independence Day
2011-09-05 Labor Day
2011-10-10 Columbus Day
2011-11-11 Veterans Day
2011-11-24 Thanksgiving
2011-12-25 Christmas Day
2011-12-26 Christmas Day (Observed)


In [16]:
# Exploring USA's national holidays on 2012
for date, name in sorted(holidays.US(state='CO', years=2012).items()):
    print(date, name)

2012-01-01 New Year's Day
2012-01-02 New Year's Day (Observed)
2012-01-16 Martin Luther King Jr. Day
2012-02-20 Washington's Birthday
2012-05-28 Memorial Day
2012-07-04 Independence Day
2012-09-03 Labor Day
2012-10-08 Columbus Day
2012-11-11 Veterans Day
2012-11-12 Veterans Day (Observed)
2012-11-22 Thanksgiving
2012-12-25 Christmas Day


In [17]:
# Exploring USA's national holidays on 2013
for date, name in sorted(holidays.US(state='CO', years=2013).items()):
    print(date, name)

2013-01-01 New Year's Day
2013-01-21 Martin Luther King Jr. Day
2013-02-18 Washington's Birthday
2013-05-27 Memorial Day
2013-07-04 Independence Day
2013-09-02 Labor Day
2013-10-14 Columbus Day
2013-11-11 Veterans Day
2013-11-28 Thanksgiving
2013-12-25 Christmas Day


In [18]:
import datetime as dt

# Selecting the most important holidays from 2011-09-01 to 2013-03-05 (and the period of time surrounding them)
important_us_holidays = ['2011-11-23', '2011-11-24', '2011-11-25', '2011-11-26', '2011-11-27',
                         '2011-12-24', '2011-12-25', '2011-12-26', '2011-12-27', '2011-12-28', '2011-12-29',
                         '2011-12-30', '2011-12-31', '2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04', '2012-01-05',
                         '2012-11-21', '2012-11-22', '2012-11-23', '2012-11-24', '2012-11-25',
                         '2012-12-24', '2012-12-25', '2012-12-26', '2012-12-27', '2012-12-28',
                         '2012-12-29', '2012-12-30',
                         '2012-12-31', '2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04']
important_us_holidays = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in important_us_holidays]

# Adding whether or not the day is an important holiday
external_data["is_important"] = 0
k = external_data.columns.get_loc("is_important")

for i in range(n): 
    date = external_data.iloc[i, 0]
    if date in important_us_holidays:
        external_data.iloc[i,k] = 1

external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,GDP,is_State_Holiday,Distance_To_Closest_Holiday,is_important
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427,92389023,42137,0,4,0
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427,92389023,42137,0,3,0
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427,92389023,42137,0,2,0
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427,92389023,42137,0,1,0
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427,92389023,42137,1,0,0


In [19]:
# Adding whether or not the day is in a weekend

external_data["is_WeekEnd"] = 0
k = external_data.columns.get_loc("is_WeekEnd")

for i in range(n): 
    date = external_data.iloc[i, 0]
    weekday = date.weekday()
    if weekday in [4, 5, 6]:
        external_data.iloc[i,k] = 1

external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,GDP,is_State_Holiday,Distance_To_Closest_Holiday,is_important,is_WeekEnd
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427,92389023,42137,0,4,0,0
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427,92389023,42137,0,3,0,1
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427,92389023,42137,0,2,0,1
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427,92389023,42137,0,1,0,1
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427,92389023,42137,1,0,0,0


In [20]:
# Adding whether or not the day is during summer

external_data["is_Summer"] = 0
k = external_data.columns.get_loc("is_Summer")

for i in range(n): 
    date = external_data.iloc[i, 0]
    month = date.month
    if month in [6, 7, 8]:
        external_data.iloc[i,k] = 1

external_data.head()

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,GDP,is_State_Holiday,Distance_To_Closest_Holiday,is_important,is_WeekEnd,is_Summer
0,2011-09-01,ATL,35,18,79,129,33.64,-84.427,92389023,42137,0,4,0,0,0
1,2011-09-02,ATL,36,15,61,185,33.64,-84.427,92389023,42137,0,3,0,1,0
2,2011-09-03,ATL,35,16,64,147,33.64,-84.427,92389023,42137,0,2,0,1,0
3,2011-09-04,ATL,27,19,93,139,33.64,-84.427,92389023,42137,0,1,0,1,0
4,2011-09-05,ATL,26,22,94,149,33.64,-84.427,92389023,42137,1,0,0,0,0


In [21]:
# Converting our external_data to a CSV file

external_data.to_csv('external_data.csv', index = None, header=True)

In [22]:
external_data

,DateOfDeparture,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,WindDirDegrees,Latitude,Longitude,Traffic,GDP,is_State_Holiday,Distance_To_Closest_Holiday,is_important,is_WeekEnd,is_Summer
0,2011-09-01,ATL,35,18,79,129,33.640,-84.427,92389023,42137,0,4,0,0,0
1,2011-09-02,ATL,36,15,61,185,33.640,-84.427,92389023,42137,0,3,0,1,0
2,2011-09-03,ATL,35,16,64,147,33.640,-84.427,92389023,42137,0,2,0,1,0
3,2011-09-04,ATL,27,19,93,139,33.640,-84.427,92389023,42137,0,1,0,1,0
4,2011-09-05,ATL,26,22,94,149,33.640,-84.427,92389023,42137,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,-3,76,320,40.777,-73.872,26722183,62605,0,11,0,1,0
11036,2013-03-02,LGA,4,-5,82,317,40.777,-73.872,26722183,62605,0,12,0,1,0
11037,2013-03-03,LGA,4,-8,69,314,40.777,-73.872,26722183,62605,0,13,0,1,0
11038,2013-03-04,LGA,5,-8,63,313,40.777,-73.872,26722183,62605,0,14,0,0,0


In [23]:
# Importing the initial data given in the starting kit
data = pd.read_csv(
    os.path.join('data', 'train.csv.bz2')
)

import problem
X, y = problem.get_train_data()

In [24]:
from sklearn.preprocessing import FunctionTransformer
from geopy import distance

def _merge_external_data(X):
    
    X = X.copy()  # Working on a copy of X to avoid modifying the original one
    X.loc[:, "DateOfDeparture"] = pd.to_datetime(X['DateOfDeparture']) # Making sure DateOfDeparture is of type DateTime
    
    # Merging our external data to have more information on the spatiotemporal setting of the departure and arrival of each flight
    columns = external_data.columns
    external_data.columns = ['DateOfDeparture', 'Departure'] + ['dep_' + col for col in columns[2:]]
    X_merged = pd.merge(X, external_data, how='left', on=['DateOfDeparture', 'Departure'], sort=False)
    external_data.columns = ['DateOfDeparture', 'Arrival'] + ['arr_' + col for col in columns[2:]]
    X_merged = pd.merge(X_merged, external_data, how='left', on=['DateOfDeparture', 'Arrival'], sort=False)

    # Dropping the duplicate columns since they're the same for departure and arrival
    X_merged = X_merged.rename(columns={'arr_is_WeekEnd':'is_WeekEnd', 'arr_is_important':'is_important',
                                        'arr_is_Summer':'is_Summer'})
    X_merged.drop(columns=['dep_is_WeekEnd', 'dep_is_important', 'dep_is_Summer'], inplace=True)
    
    # Computing the distance between the two airports of each flight
    X_merged['Distance'] = X_merged.apply(lambda x : distance.distance((x['dep_Latitude'], x['dep_Longitude']),
                                                                       (x['arr_Latitude'], x['arr_Longitude'])).km, axis=1)
    
    # Adding product columns to represent the interaction between some departure/arrival pairs of variables
    X_merged["inter_Traffic"] = X_merged.loc[:, "dep_Traffic"] * X_merged.loc[:, "arr_Traffic"]
    X_merged["inter_GDP"] = X_merged.loc[:, "dep_GDP"] * X_merged.loc[:, "arr_GDP"]
    X_merged["inter_StateHoliday"] = X_merged.loc[:, "dep_is_State_Holiday"] * X_merged.loc[:, "arr_is_State_Holiday"]
    X_merged["inter_DistanceToClosestHoliday"] = X_merged.loc[:, "dep_Distance_To_Closest_Holiday"] * X_merged.loc[:, "arr_Distance_To_Closest_Holiday"]
    
    return X_merged

data_merger = FunctionTransformer(_merge_external_data)

In [25]:
merged = data_merger.fit_transform(X)
merged

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,dep_Max TemperatureC,dep_MeanDew PointC,dep_Max Humidity,dep_WindDirDegrees,dep_Latitude,dep_Longitude,dep_Traffic,dep_GDP,dep_is_State_Holiday,dep_Distance_To_Closest_Holiday,arr_Max TemperatureC,arr_MeanDew PointC,arr_Max Humidity,arr_WindDirDegrees,arr_Latitude,arr_Longitude,arr_Traffic,arr_GDP,arr_is_State_Holiday,arr_Distance_To_Closest_Holiday,is_important,is_WeekEnd,is_Summer,Distance,inter_Traffic,inter_GDP,inter_StateHoliday,inter_DistanceToClosestHoliday
0,2012-06-19,ORD,DFW,12.875000,9.812647,35,18,62,208,41.979,-87.904,66629600,52154,0,15,34,21,82,161,32.896,-97.037,58620160,50231,1,0,0,0,1,1290.440811,3905837812736000,2619747574,0,0
1,2012-09-10,LAS,DEN,14.285714,9.466734,36,13,51,187,36.080,-115.152,40799830,43345,0,7,33,-6,21,207,39.858,-104.667,53156278,49622,0,7,0,0,0,1011.381809,2168767105832740,2150865590,0,49
2,2012-10-05,DEN,LAX,10.863636,9.035883,6,-2,92,357,39.858,-104.667,53156278,49622,0,3,22,16,93,266,33.942,-118.408,63688121,52912,0,3,0,1,0,1387.356010,3385423465173638,2625599264,0,9
3,2011-10-09,ATL,ORD,11.480000,7.990202,22,10,67,70,33.640,-84.427,92389023,42137,0,1,27,10,83,93,41.979,-87.904,66701241,51191,0,1,0,1,0,974.657770,6162462488877543,2157035167,0,1
4,2012-02-21,DEN,SFO,11.450000,9.517159,8,-9,46,270,39.858,-104.667,53156278,49622,0,1,16,8,93,300,37.619,-122.375,44399885,52912,0,1,0,0,0,1556.878735,2360132630228030,2625599264,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8897,2011-10-02,DTW,ATL,9.263158,7.316967,17,2,85,327,42.212,-83.353,32406159,39756,0,8,20,2,76,329,33.640,-84.427,92389023,42137,0,8,0,1,0,956.103843,2993973369192657,1675198572,0,64
8898,2012-09-25,DFW,ORD,12.772727,10.641034,35,16,79,187,32.896,-97.037,58620160,50231,0,13,25,11,93,216,41.979,-87.904,66629600,52154,0,13,0,0,0,1290.440811,3905837812736000,2619747574,0,169
8899,2012-01-19,SFO,LAS,11.047619,7.908705,11,5,93,125,37.619,-122.375,44399885,52912,0,3,13,-9,40,197,36.080,-115.152,40799830,43345,0,3,0,0,0,666.253106,1811507760019550,2293470640,0,9
8900,2013-02-03,ORD,PHL,6.076923,4.030334,-6,-12,91,268,41.979,-87.904,66777161,51982,0,9,1,-6,92,296,39.872,-75.241,30504112,48380,0,13,0,1,0,1090.862923,2036977998186032,2514889160,0,117


In [26]:
# import seaborn as sns

# corr = merged.corr()
# plt.figure(figsize=(40, 40))
# sns.heatmap(corr, center=0, annot=True, cmap='RdYlBu');
# #plt.savefig('heatmap.png')

In [27]:
def _encode_dates(X):
    
    # Encoding the date information from the DateOfDeparture column
    X.loc[:, 'year'] = X['DateOfDeparture'].dt.year
    X.loc[:, 'month'] = X['DateOfDeparture'].dt.month
    X.loc[:, 'day'] = X['DateOfDeparture'].dt.day
    X.loc[:, 'weekday'] = X['DateOfDeparture'].dt.weekday
    X.loc[:, 'week'] = X['DateOfDeparture'].dt.isocalendar().week
    X.loc[:, 'n_days'] = X['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)
    
    # Making the date information binary
    date_info = ['year', 'month', 'day', 'weekday', 'week']
    for feat in date_info:
        X = X.join(pd.get_dummies(X[feat], prefix=feat))
    
    # Finally, we can drop the original DateOfDeparture column from the dataframe
    return X.drop(columns=["DateOfDeparture"])

date_encoder = FunctionTransformer(_encode_dates)

In [28]:
df_final = date_encoder.fit_transform(merged)
df_final

,Departure,Arrival,WeeksToDeparture,std_wtd,dep_Max TemperatureC,dep_MeanDew PointC,dep_Max Humidity,dep_WindDirDegrees,dep_Latitude,dep_Longitude,dep_Traffic,dep_GDP,dep_is_State_Holiday,dep_Distance_To_Closest_Holiday,arr_Max TemperatureC,arr_MeanDew PointC,arr_Max Humidity,arr_WindDirDegrees,arr_Latitude,arr_Longitude,arr_Traffic,arr_GDP,arr_is_State_Holiday,arr_Distance_To_Closest_Holiday,is_important,is_WeekEnd,is_Summer,Distance,inter_Traffic,inter_GDP,inter_StateHoliday,inter_DistanceToClosestHoliday,year,month,day,weekday,week,n_days,year_2011,year_2012,year_2013,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14,week_15,week_16,week_17,week_18,week_19,week_20,week_21,week_22,week_23,week_24,week_25,week_26,week_27,week_28,week_29,week_30,week_31,week_32,week_33,week_34,week_35,week_36,week_37,week_38,week_39,week_40,week_41,week_42,week_43,week_44,week_45,week_46,week_47,week_48,week_49,week_50,week_51,week_52
0,ORD,DFW,12.875000,9.812647,35,18,62,208,41.979,-87.904,66629600,52154,0,15,34,21,82,161,32.896,-97.037,58620160,50231,1,0,0,0,1,1290.440811,3905837812736000,2619747574,0,0,2012,6,19,1,25,15510,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,LAS,DEN,14.285714,9.466734,36,13,51,187,36.080,-115.152,40799830,43345,0,7,33,-6,21,207,39.858,-104.667,53156278,49622,0,7,0,0,0,1011.381809,2168767105832740,2150865590,0,49,2012,9,10,0,37,15593,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,DEN,LAX,10.863636,9.035883,6,-2,92,357,39.858,-104.667,53156278,49622,0,3,22,16,93,266,33.942,-118.408,63688121,52912,0,3,0,1,0,1387.356010,3385423465173638,2625599264,0,9,2012,10,5,4,40,15618,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,ATL,ORD,11.480000,7.990202,22,10,67,70,33.640,-84.427,92389023,42137,0,1,27,10,83,93,41.979,-87.904,66701241,51191,0,1,0,1,0,974.657770,6162462488877543,2157035167,0,1,2011,10,9,6,40,15256,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,DEN,SFO,11.450000,9.517159,8,-9,46,270,39.858,-104.667,53156278,49622,0,1,16,8,93,300,37.619,-122.375,44399885,52912,0,1,0,0,0,1556.878735,2360132630228030,2625599264,0,1,2012,2,21,1,8,15391,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8897,DTW,ATL,9.263158,7.316967,17,2,85,32

In [29]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import lightgbm as lgb

categorical_encoder = make_pipeline(SimpleImputer(strategy='constant', fill_value='missing'), OneHotEncoder())
    
categorical_cols = ['Arrival', 'Departure']

preprocessor = make_column_transformer((categorical_encoder, categorical_cols), remainder='passthrough')

regressor = lgb.LGBMRegressor(boosting_type='dart', n_estimators=10000, learning_rate=0.1, max_depth=-1,
                              num_leaves=16, subsample=0.9, colsample_bytree=0.9, subsample_freq=1,uniform_drop=True)

pipeline = make_pipeline(data_merger, date_encoder, preprocessor, regressor)

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

merger = pipeline[0]
X_train_augmented = merger.transform(X_train)
X_test_augmented = merger.transform(X_test)

predictor = pipeline[1:]
predictor.fit(X_train_augmented, y_train).score(X_test_augmented, y_test)

KeyError: "['dep_is_WeekEnd' 'dep_is_important' 'dep_is_Summer'] not found in axis"

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import lightgbm as lgb
from sklearn.pipeline import make_pipeline, Pipeline
    
categorical_cols = ['Arrival', 'Departure']
    
#preprocessor = make_column_transformer((OneHotEncoder(), categorical_cols), remainder='passthrough')

data = X.copy()
data = data.join(pd.get_dummies(data.loc[:,'Departure'], prefix='dep'))
data = data.join(pd.get_dummies(data.loc[:,'Arrival'], prefix='arr'))
data = data.drop('Departure', axis=1)
data = data.drop('Arrival', axis=1)
data.loc[:, "DateOfDeparture"] = pd.to_datetime(data['DateOfDeparture'])
data.loc[:, 'year'] = data['DateOfDeparture'].dt.year
data.loc[:, 'month'] = data['DateOfDeparture'].dt.month
data.loc[:, 'day'] = data['DateOfDeparture'].dt.day
data.loc[:, 'weekday'] = data['DateOfDeparture'].dt.weekday
data.loc[:, 'week'] = data['DateOfDeparture'].dt.isocalendar().week
data.loc[:, 'n_days'] = data['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

# They're categorical features, so we make them binary
features = ['year', 'month', 'day', 'weekday', 'week']
for feat in features:
    data = data.join(pd.get_dummies(data[feat], prefix=feat))
    
data.drop(columns=["DateOfDeparture"], inplace=True)

regressor = lgb.LGBMRegressor(boosting_type='dart', n_estimators=10, subsample_freq=1, max_bin=511, uniform_drop=True)
#learning_rate=0.1, max_depth=5, num_leaves=16, subsample=0.9, subsample_freq=1, colsample_bytree = 0.9, max_bin=511, uniform_drop=True

param_grid = {
    'lgb__uniform_drop': [True, False]}
#     'lgb__learning_rate': np.arange(0.1,1.1,0.1),
#     'lgb__max_depth': np.arange(-1,11),
#     'lgb__num_leaves': np.arange(10,32),
#     'lgb__subsample': np.arange(0.1,1.1,0.1),
#     'lgb__colsample_bytree': np.arange(0.1,1.1,0.1),
# }

#     'lgb__subsample_freq': np.arange(1,6),
#     'lgb__max_bin': [255, 511, 1023],
#     'lgb__uniform_drop': [True, False],   

search = GridSearchCV(lgb.LGBMRegressor(), param_grid, scoring='accuracy', verbose =5, n_jobs=-1, cv=2)
search.fit(data, y)


Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.1s finished


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: week

In [35]:
type(data['week'][0])

numpy.uint32

In [ ]:
# Final version : 0.329/0.338
#! ramp-test --submission lgb

In [ ]:
# Final version : 0.329/0.338
! ramp-test --submission lgb

Testing Number of air passengers prediction
Reading train and test files from ./data ...
Reading cv ...
Training submissions/lgb ...
CV fold 0
	score   rmse       time
	train  0.005  64.522055
	valid  0.357   6.344650
	test   0.365   1.631402
CV fold 1
	score   rmse       time
	train  0.005  67.326404
	valid  0.352   6.281847
	test   0.357   1.683833
CV fold 2
	score   rmse       time
	train  0.005  61.214797
	valid  0.356   6.382746
	test   0.368   1.661717
CV fold 3
	score   rmse       time
	train  0.005  61.691296
	valid  0.362   6.360367
	test   0.367   1.636719
CV fold 4
	score   rmse       time
	train  0.005  61.097506
	valid  0.350   6.351507
	test   0.351   1.693951
CV fold 5
	score   rmse       time
	train  0.005  62.157972
	valid  0.343   6.444071
	test   0.353   1.649975
CV fold 6
	score   rmse       time
	train  0.005  67.009062
	valid  0.348   6.380219
	test   0.348   1.630078
CV fold 7
